In [4]:
import pandas as pd
import regex as re
import requests

In [3]:
recipes = pd.read_json("../input/bbc_food.json")

## First, we analyze the dataset to check for null values or invalid formats

In [4]:
recipes.head()

,total_ratings,serving,recipe_image,prep_time,ingredients,url,title,uniq_id,source,cooking_time,chef_url,skill_level,crawled_at,prep_methods,chef_name
0,1789 ratings,Serves 4,www.bbcgoodfood.com/sites/default/files/styles...,10 mins,"[1 large onion, 1 red pepper, 2 garlic cloves,...",https://www.bbcgoodfood.com/recipes/chilli-con...,Chilli con carne recipe,c2102009-5c5c-598a-8bfb-6af3f569490f,https://www.bbcgoodfood.com,1 hr,https://www.bbcgoodfood.com/publication/good-f...,Easy,2020-03-29 01:29:15,[Prepare your vegetables. Chop 1 large onion i...,Good Food
1,679 ratings,Serves 4,www.bbcgoodfood.com/sites/default/files/styles...,na,"[1 tbsp olive oil, 4 boneless skinless chicken...",https://www.bbcgoodfood.com/recipes/summer-win...,Summer-in-winter chicken,5b1dd6f5-0088-5b2a-9f1b-712b8e084607,https://www.bbcgoodfood.com,na,https://www.bbcgoodfood.com/chef/mary-cadogan,Easy,2020-03-29 01:29:15,"[Heat the oil in a frying pan, preferably non-...",Mary Cadogan
2,971 ratings,Serves 4,www.bbcgoodfood.com/sites/default/files/styles...,10 mins,"[2 tsp cumin seeds, pinch chilli flakes, 2 tbs...",https://www.bbcgoodfood.com/recipes/spiced-car...,Spiced carrot & lentil soup,742cf11a-4a39-5e1e-8248-85c58cb10a2c,https://www.bbcgoodfood.com,15 mins,https://www.bbcgoodfood.com/chef/jane-hornby,Easy,2020-03-29 01:29:15,[Heat a large saucepan and dry-fry 2 tsp cumin...,Jane Hornby
3,755 ratings,Serves 4,www.bbcgoodfood.com/sites/default/files/styles...,10 mins,"[2 tbsp sunflower or vegetable oil, 1 onion, c...",https://www.bbcgoodfood.com/recipes/spicy-root...,Spicy root & lentil casserole,b3fd81b2-7906-55ee-bccb-d0699f7076d9,https://www.bbcgoodfood.com,35 mins,https://www.bbcgoodfood.com/publication/good-f...,Easy,2020-03-29 01:29:15,[Heat the oil in a large pan and cook the onio...,Good Food
4,697 ratings,Serves 4,www.bbcgoodfood.com/sites/default/files/styles...,10 mins,"[2 tsp cumin seeds, large pinch chilli flakes,...",https://www.bbcgoodfood.com/recipes/red-lentil...,"Red lentil, chickpea & chilli soup",540b1b89-ba24-55bb-a290-2c2d0db5757e,https://www.bbcgoodfood.com,25 mins,https://www.bbcgoodfood.com/publication/good-f...,Easy,2020-03-29 01:29:16,[Heat a large saucepan and dry-fry 2 tsp cumin...,Good Food


In [4]:
recipes["url"][1]

'https://www.bbcgoodfood.com/recipes/summer-winter-chicken'

In [5]:
recipes["ingredients"][0]

['1 large onion',
 '1 red pepper',
 '2 garlic cloves',
 '1 tbsp oil',
 '1 heaped tsp hot chilli powder (or 1 level tbsp if you only have mild)',
 '1 tsp paprika',
 '1 tsp ground cumin',
 '500g lean minced beef',
 '1 beef stock cube',
 '400g can chopped tomatoes',
 '½ tsp dried marjoram',
 '1 tsp sugar (or add a thumbnail-sized piece of dark chocolate along with the beans instead, see tip)',
 '2 tbsp tomato purée',
 '410g can red kidney beans',
 'plain boiled long grain rice, to serve',
 'soured cream, to serve']

In [9]:
list_ingred = list((x for l in recipes["ingredients"] for x in l))

In [10]:
len(list_ingred)

123412

In [11]:
unique_ingred = set(x for l in recipes["ingredients"] for x in l)

In [12]:
len(unique_ingred)

60484

In [ ]:
def extractimages(a):
    """This function obtains the main image of each recipe
    Input = URL of the recipe
    Output= Image of the recipe
    """
    resp = requests.get(f"{a}")
    soup = BeautifulSoup(resp.text, 'html.parser')
    tag = soup.find_all("img")[2]
    b = tag["src"]
    return b
    

In [ ]:
recipes["new_images"]=[extractimages(a) for a in recipes["url"]]

In [ ]:
recipes = recipes.drop("new_images",axis=1)

In [ ]:
for a in recipes["url"][:5]:
    res = requests.get(f"{a}")
    soup = BeautifulSoup(res.text, 'html.parser')
    tag = soup.find_all("img")[2]
    print(tag["src"])

In [ ]:
def extractsummary():
    for a in recipes["url"][:5]:
        res = requests.get(f"{a}")
        soup = BeautifulSoup(res.text, 'html.parser')
        tag = soup.find_all("p")[0]
        b = tag["src"]
        return b

In [ ]:
def preptime(a):
    """This function obtains the preparation time of each recipe
    Input = URL of the recipe
    Output= Time of preparation, e.g = "10 mins"
    """
    res = requests.get(f'{a}')
    soup = BeautifulSoup(res.text, 'html.parser')
    tag = soup.find_all("time")
    tag = str(tag)
    a  = re.findall(r"\d\d\ \w+",tag)
    try:
        return (a[0])
    except: 
        return "Unknown"

In [ ]:
recipes["new_prep_time"]=[preptime(a) for a in recipes["url"]]


In [ ]:
recipes["new_prep_time"].value_counts()

In [ ]:
recipes_clean = recipes.drop(["prep_time","cooking_time"],axis=1)

## After all the cleanup, here we have the final dataset:

In [5]:
recipes_clean = pd.read_json("../input/recipes.json")

In [6]:
recipes_clean

,total_ratings,serving,recipe_image,ingredients,url,title,uniq_id,source,chef_url,skill_level,crawled_at,prep_methods,chef_name,new_images,new_prep_time
0,1789 ratings,Serves 4,www.bbcgoodfood.com/sites/default/files/styles...,"[1 large onion, 1 red pepper, 2 garlic cloves,...",https://www.bbcgoodfood.com/recipes/chilli-con...,Chilli con carne recipe,c2102009-5c5c-598a-8bfb-6af3f569490f,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/publication/good-f...,Easy,2020-03-29 01:29:15,[Prepare your vegetables. Chop 1 large onion i...,Good Food,https://images.immediate.co.uk/production/vola...,10 mins
1,679 ratings,Serves 4,www.bbcgoodfood.com/sites/default/files/styles...,"[1 tbsp olive oil, 4 boneless skinless chicken...",https://www.bbcgoodfood.com/recipes/summer-win...,Summer-in-winter chicken,5b1dd6f5-0088-5b2a-9f1b-712b8e084607,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/chef/mary-cadogan,Easy,2020-03-29 01:29:15,"[Heat the oil in a frying pan, preferably non-...",Mary Cadogan,https://images.immediate.co.uk/production/vola...,25 mins
2,971 ratings,Serves 4,www.bbcgoodfood.com/sites/default/files/styles...,"[2 tsp cumin seeds, pinch chilli flakes, 2 tbs...",https://www.bbcgoodfood.com/recipes/spiced-car...,Spiced carrot & lentil soup,742cf11a-4a39-5e1e-8248-85c58cb10a2c,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/chef/jane-hornby,Easy,2020-03-29 01:29:15,[Heat a large saucepan and dry-fry 2 tsp cumin...,Jane Hornby,https://images.immediate.co.uk/production/vola...,10 mins
3,755 ratings,Serves 4,www.bbcgoodfood.com/sites/default/files/styles...,"[2 tbsp sunflower or vegetable oil, 1 onion, c...",https://www.bbcgoodfood.com/recipes/spicy-root...,Spicy root & lentil casserole,b3fd81b2-7906-55ee-bccb-d0699f7076d9,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/publication/good-f...,Easy,2020-03-29 01:29:15,[Heat the oil in a large pan and cook the onio...,Good Food,https://images.immediate.co.uk/production/vola...,10 mins
4,697 ratings,Serves 4,www.bbcgoodfood.com/sites/default/files/styles...,"[2 tsp cumin seeds, large pinch chilli flakes,...",https://www.bbcgoodfood.com/recipes/red-lentil...,"Red lentil, chickpea & chilli soup",540b1b89-ba24-55bb-a290-2c2d0db5757e,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/publication/good-f...,Easy,2020-03-29 01:29:16,[Heat a large saucepan and dry-fry 2 tsp cumin...,Good Food,https://images.immediate.co.uk/production/vola...,10 mins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12955,0 ratings,"Serves 1, plus 1 lunchbox",www.bbcgoodfood.com/sites/default/files/styles...,"[3 tsp olive oil, 1 orange, zested and juiced,...",https://www.bbcgoodfood.com/recipes/salsa-verd...,Salsa verde salmon with smashed chickpea salad,13ce6ed6-e9a8-5dca-87bd-d26de6936d8c,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/writer/anna-glover,Easy,2020-03-29 02:00:56,[Heat the grill to high. Whisk 1 tsp of the oi...,Anna Glover,https://images.immediate.co.uk/production/vola...,15 mins
12956,0 ratings,"Serves 4, plus 2 lunchboxes",www.bbcgoodfood.com/sites/default/files/styles...,"[250g plain flour, plus extra for dusting, 2 t...",https://www.bbcgoodfood.com/recipes/chicken-sw...,Chicken & sweetcorn tacos,a909b49a-4fb7-54e9-a4e9-d1ca6087a7ab,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/writer/anna-glover,Easy,2020-03-29 02:00:56,[Combine the flour with half the oil and a sma...,Anna Glover,https://images.immediate.co.uk/production/vola...,30 mins
12957,0 ratings,"Serves 2, plus 2 lunchboxes",www.bbcgoodfood.com/sites/default/files/styles...,"[1 tbsp olive oil, plus 1 tsp, 3 slices pancet...",https://www.bbcgoodfood.com/recipes/prawn-panc...,"Prawn, pancetta & watercress risotto",3ffe356a-d5ea-5b8c-b30c-a23f0bcdacc6,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/writer/anna-glover,Easy,2020-03-29 02:00:56,[Heat 1 tbsp oil in a wide pan and fry the pan...,Anna Glover,https://images.immediate.co.uk/production/vola...,30 mins


In [26]:
recipes_clean.iloc[5]

total_ratings                                          513 ratings
serving                                                  Serves 4 
recipe_image     www.bbcgoodfood.com/sites/default/files/styles...
ingredients      [9 dried lasagne sheets, 1 tbsp sunflower oil,...
url              https://www.bbcgoodfood.com/recipes/creamy-cou...
title                                     Creamy courgette lasagne
uniq_id                       40a83d9f-8063-5120-b8a6-5f1f007eb687
source                                 https://www.bbcgoodfood.com
chef_url         https://www.bbcgoodfood.com/publication/good-f...
skill_level                                                  Easy 
crawled_at                                     2020-03-29 01:29:16
prep_methods     [Heat oven to 220C/fan 200C/gas 7. Put a pan o...
chef_name                                                Good Food
new_images       https://images.immediate.co.uk/production/vola...
new_prep_time                                              10 

## Function to show images

In [17]:
# Importing Required Modules
import sys
import requests
from PIL import Image
# Exception Handling for invalid requests
try:
        # Creating an request object to store the response
        # The URL is refrenced sys.argv[1]
    ImgRequest = requests.get('https://images.immediate.co.uk/production/volatile/sites/30/2020/08/recipe-image-legacy-id-1273575_8-607ddec.jpg?quality=90&resize=440,400'
)
        # Verifying whether the specified URL exist or not
    if ImgRequest.status_code == requests.codes.ok:
        
                # Opening a file to write bytes from response content
                # Storing this onject as an image file on the hard drive
        img = open("test.jpg","wb")
        img.write(ImgRequest.content)
        img.close()
                # Opening Inage file using PIL Image
        img = Image.open("test.jpg")
        img.show()
    else:
        print(ImgRequest.status_code)
except Exception as e:
    print(str(e))

### Export to json

In [10]:
recipes_clean = recipes_clean.drop("recipe_image",axis=1)

In [11]:
recipes_clean

,total_ratings,serving,ingredients,url,title,uniq_id,source,chef_url,skill_level,crawled_at,prep_methods,chef_name,new_images,new_prep_time
0,1789 ratings,Serves 4,"[1 large onion, 1 red pepper, 2 garlic cloves,...",https://www.bbcgoodfood.com/recipes/chilli-con...,Chilli con carne recipe,c2102009-5c5c-598a-8bfb-6af3f569490f,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/publication/good-f...,Easy,2020-03-29 01:29:15,[Prepare your vegetables. Chop 1 large onion i...,Good Food,https://images.immediate.co.uk/production/vola...,10 mins
1,679 ratings,Serves 4,"[1 tbsp olive oil, 4 boneless skinless chicken...",https://www.bbcgoodfood.com/recipes/summer-win...,Summer-in-winter chicken,5b1dd6f5-0088-5b2a-9f1b-712b8e084607,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/chef/mary-cadogan,Easy,2020-03-29 01:29:15,"[Heat the oil in a frying pan, preferably non-...",Mary Cadogan,https://images.immediate.co.uk/production/vola...,25 mins
2,971 ratings,Serves 4,"[2 tsp cumin seeds, pinch chilli flakes, 2 tbs...",https://www.bbcgoodfood.com/recipes/spiced-car...,Spiced carrot & lentil soup,742cf11a-4a39-5e1e-8248-85c58cb10a2c,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/chef/jane-hornby,Easy,2020-03-29 01:29:15,[Heat a large saucepan and dry-fry 2 tsp cumin...,Jane Hornby,https://images.immediate.co.uk/production/vola...,10 mins
3,755 ratings,Serves 4,"[2 tbsp sunflower or vegetable oil, 1 onion, c...",https://www.bbcgoodfood.com/recipes/spicy-root...,Spicy root & lentil casserole,b3fd81b2-7906-55ee-bccb-d0699f7076d9,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/publication/good-f...,Easy,2020-03-29 01:29:15,[Heat the oil in a large pan and cook the onio...,Good Food,https://images.immediate.co.uk/production/vola...,10 mins
4,697 ratings,Serves 4,"[2 tsp cumin seeds, large pinch chilli flakes,...",https://www.bbcgoodfood.com/recipes/red-lentil...,"Red lentil, chickpea & chilli soup",540b1b89-ba24-55bb-a290-2c2d0db5757e,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/publication/good-f...,Easy,2020-03-29 01:29:16,[Heat a large saucepan and dry-fry 2 tsp cumin...,Good Food,https://images.immediate.co.uk/production/vola...,10 mins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12955,0 ratings,"Serves 1, plus 1 lunchbox","[3 tsp olive oil, 1 orange, zested and juiced,...",https://www.bbcgoodfood.com/recipes/salsa-verd...,Salsa verde salmon with smashed chickpea salad,13ce6ed6-e9a8-5dca-87bd-d26de6936d8c,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/writer/anna-glover,Easy,2020-03-29 02:00:56,[Heat the grill to high. Whisk 1 tsp of the oi...,Anna Glover,https://images.immediate.co.uk/production/vola...,15 mins
12956,0 ratings,"Serves 4, plus 2 lunchboxes","[250g plain flour, plus extra for dusting, 2 t...",https://www.bbcgoodfood.com/recipes/chicken-sw...,Chicken & sweetcorn tacos,a909b49a-4fb7-54e9-a4e9-d1ca6087a7ab,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/writer/anna-glover,Easy,2020-03-29 02:00:56,[Combine the flour with half the oil and a sma...,Anna Glover,https://images.immediate.co.uk/production/vola...,30 mins
12957,0 ratings,"Serves 2, plus 2 lunchboxes","[1 tbsp olive oil, plus 1 tsp, 3 slices pancet...",https://www.bbcgoodfood.com/recipes/prawn-panc...,"Prawn, pancetta & watercress risotto",3ffe356a-d5ea-5b8c-b30c-a23f0bcdacc6,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/writer/anna-glover,Easy,2020-03-29 02:00:56,[Heat 1 tbsp oil in a wide pan and fry the pan...,Anna Glover,https://images.immediate.co.uk/production/vola...,30 mins
12958,0 ratings,"Serves 2, plus 2 lunchboxes","[½ tbsp groundnut oil, 100g lean pork mince, 1...",https://www.bbcgoodfood.com/recipes/pork-auber...,Pork & aubergine noodle stir-fry,8d058552-4e3f-5010-a70a-f40d32ad5ba2,https://www.bbcgoodfood.com,https://www.bbcgoodfood.com/writer/anna-glover,Easy,2020-03-29 02:00:56,[Heat the oil in a wok or high-sided frying pa...,Anna Glover,https://images.immediate.co.uk/produc

In [12]:
recipes_clean.to_json("../input/recipes1.json",orient="records")

In [13]:
# Imported to MongoDB through the console:
# mongoimport -d fridge -c recipes --file recipes1.json  --jsonArray 

### Mongo Queries

In [15]:
# {ingredients:{$regex:"spaghetti"}}

In [16]:
recipes_clean["skill_level"].value_counts()

 Easy            11006
 More effort      1705
 A challenge       249
Name: skill_level, dtype: int64

In [4]:
from database import db
projection = {"title":1,"prep_methods":1}

for x in db.recipes.find({"ingredients":{"$regex":"onion"}},projection):
    print(x)

{'_id': ObjectId('5f806a0ccbc753df1c095cf7'), 'title': 'Spicy root & lentil casserole', 'prep_methods': ['Heat the oil in a large pan and cook the onion and garlic over a medium heat for 3-4 minutes until softened, stirring occasionally. Tip in the potatoes, carrots and parsnips, turn up the heat and cook for 6-7 minutes, stirring, until the vegetables are golden.', 'Stir in the curry paste or powder, pour in the stock and then bring to the boil. Reduce the heat, add the lentils, cover and simmer for 15-20 minutes until the lentils and vegetables are tender and the sauce has thickened.', 'Stir in most of the coriander, season and heat for a minute or so.Top with yogurt and the rest of the coriander. Serve with naan bread.']}
{'_id': ObjectId('5f806a0ccbc753df1c095cfa'), 'title': 'Chilli con carne recipe', 'prep_methods': ['Prepare your vegetables.\xa0Chop\xa01 large onion into small dice, about 5mm square. The easiest way to do this is to cut the onion in half from root to tip, peel it

{'_id': ObjectId('5f806a0ccbc753df1c095ea4'), 'title': 'Lancashire hotpot', 'prep_methods': ['Heat oven to 160C/fan 140C/gas 3.', 'Heat a little of the 100g dripping or butter in a large shallow casserole dish and brown 900g stewing lamb chunks in batches, lift to a plate, then repeat with 3 trimmed and sliced lamb kidneys.', 'Fry 2 chopped onions and 4 peeled and sliced carrots in the pan with a little more dripping until golden.', 'Sprinkle over 25g plain flour, allow to cook for a couple of mins, shake over 2 tsp Worcestershire sauce, pour in 500ml lamb or chicken stock, then bring to the boil.', 'Stir in the stewing lamb and kidneys and 2 bay leaves, then turn off the heat.', 'Arrange 900g peeled and sliced potatoes on top of the meat, then drizzle with a little more dripping.', 'Cover, then place in the oven for about 1½ hrs until the potatoes are cooked.', 'Remove the lid, brush the potatoes with a little more dripping, then turn the oven up to brown the potatoes, or finish under

{'_id': ObjectId('5f806a0ccbc753df1c0968e8'), 'title': 'Chicken bhuna', 'prep_methods': ['To make the onion purée, bring a small pan of water to the boil and add half the roughly chopped onion. Boil until soft (about 10 mins), drain and puree with a hand blender or mini food processor.', "Heat a large saucepan on a high heat. Once it's hot add the oil and finely\xa0chopped onions and reduce to a low heat. Cook the onions slowly and gently until golden brown in colour, and season.", 'Add the ginger and garlic purée, curry powder, turmeric, chilli powder and a splash of water to the pan of onions and stir in well. Fry for a couple of mins. Add the diced chicken thighs and stir in well.', 'Mix the yogurt, tomato purée and onion purée together in a jug with 300ml water. Pour into the saucepan and mix well. Turn up the heat until the sauce begins to boil. Simmer for 15 to\xa020 mins, stirring occasionally.', 'Finally, sprinkle in the garam masala and stir in well for the final 2 mins before

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

